In [1]:
# pip install -U pip google-cloud-vision
# pip install opencv-contrib-python

In [2]:
# import the necessary packages
from google.oauth2 import service_account
from google.cloud import vision
import argparse
import cv2
import io
import numpy as np
import time, os
import pandas as pd

In [3]:
# export GOOGLE_APPLICATION_CREDENTIALS="ds-mcri-360405-477968334e7c.json"

In [4]:
def detect_text(path):
    """Detects text in the file."""

    each_full_detection = []

    from google.cloud import vision
    import io

    # Connect to the online service
    creds = service_account.Credentials.from_service_account_file('ds-mcri-360405-477968334e7c.json')
    client = vision.ImageAnnotatorClient(credentials=creds,)

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    # Print each detected word without line break marks
    for text in texts:
        text_content = '"{}"'.format(text.description)
        each_full_detection.append(text_content)
    return each_full_detection

    # Print cooresponding bounding box of each detected word
    #     vertices = (['({},{})'.format(vertex.x, vertex.y)
    #                 for vertex in text.bounding_poly.vertices])

    #     print('bounds: {}'.format(','.join(vertices)))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [5]:
all_detected = {}
test_images_dir = os.path.join('hpf_test')
all_image_names = sorted(os.listdir(test_images_dir))
all_image_paths = [test_images_dir+'/'+ name for name in all_image_names]
for each_path in all_image_paths:
    all_detected[each_path] = detect_text(each_path)[0].replace("\n", " ").strip() 

In [6]:
len(all_detected)

10

In [7]:
df_detexted = pd.DataFrame(all_detected.items(), columns=['ImageIndex', 'DetectedContent'])
df_detexted.to_csv("df_GCPdetexted.csv")

In [8]:
#all_detected["hpf_test/testImage01.jpg"]

In [9]:
# result_detected = {}
# result_images_dir = os.path.join('resultImage')
# image_names = sorted(os.listdir(result_images_dir))
# image_paths = [result_images_dir+'/'+ name for name in image_names]
# for each_path in image_paths:
#     result_detected[each_path] = detect_text(each_path)[0].replace("\n", " ").strip() 

In [11]:
# len(result_detected)

In [12]:
# result_df_detexted = pd.DataFrame(result_detected.items(), columns=['ImageIndex', 'DetectedContent'])
# result_df_detexted.to_csv("result_df_detexted.csv")